<h1 align=center>Clustering Neighborhoods in New York City and Toronto</h1>
<h3>Applied Data Science Capstone by IBM on Coursera</h3>

## Table of contents <a name="top"></a>

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#introduction">Introduction</a>

2. <a href="#data">Data</a>

3. <a href="#methodology">Methodology and Analysis</a>

4. <a href="#results">Results</a>

5. <a href="#discussion">Discussion</a>

6. <a href="#conclusion">Conclusion</a>
</font>
</div>

## 1. Introduction <a name="introduction"></a> &nbsp; <font size = 1>[return to top](#top)</font>

New York and Toronto are both recognized as being between the most multicultural and cosmopolitan cities in the world. They are also the most populous cities and the financial capitals of their respective countries. As such, there's a large influx of people and businesses moving into both cities every year.

Moving into a new city involves a lot of thinking and planning. One important aspect a person or a business could consider when thinking about moving is which area in the new city is similar to the region they are currently at, or to a region they are familiar with.

In this project, we explore neighborhoods in New York and Toronto and cluster them into groups of similar neighborhoods. In doing so, we aim to aid households and business looking into moving between both cities to better choose which area in the new city they might be most interested in moving to.

## 2. Data <a name="data"></a> &nbsp; <font size = 1>[return to top](#top)</font>

To compare the regions of both cities, we will group neighborhoods according to most common types of venues in each of them. First, we need a list of neighborhoods in each of the cities and their coordinates, which we already created in previous projects.

NYU Spatial Data Repository's dataset, available at https://geo.nyu.edu/catalog/nyu_2451_34572was, was used to create the dataframe of neighborhoods in New York. Data for neighborhoods in Toronto was not as readily available, so for Toronto's dataframe we scraped Wikipedia's page on Toronto postal codes (available at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) in order to obtain the names of the neighborhoods and boroughs, and then used Python's Geocoder package to obtain the coordinates of each postal code. Notice, therefore, that we will be working with neighborhoods in New York and with postal codes in Toronto (which may include more than on neighborhood); for simplicity, in the rest of the notebook we will refer to both as simply "neighborhoods".

We will then use Foursquare API to get venue information for the neighborhoods (or postal codes) of both cities, and use the venues' categories to cluster the neighborhoods (or postal codes) into groups with similar venue categories.

### Loading the dataframes

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import metrics for model evaluation
from sklearn import metrics

# map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# library to convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


Now, let's load the dataframes of neighborhodds we created previously.

In [2]:
import pandas as pd

df_ny = pd.read_csv('new_york_neighborhoods.csv')

df_ny.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [3]:
df_toronto = pd.read_csv('toronto_neighborhoods.csv')

df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Let's create a dataframe which combines both of these (with postal codes in Toronto being listed under neighborhoods).

In [4]:
# removes the 'Neighborhood' column from Toronto dataframe and renames the 'PostalCode' column
df_toronto1 = df_toronto.drop(['Neighborhood'], axis = 1).rename(columns={'PostalCode':'Neighborhood'})

# changes the order of the columns to match the New York dataframe
df_toronto1 = df_toronto1[['Borough', 'Neighborhood', 'Latitude', 'Longitude']]

# creates new dataframe combining both cities
df = df_ny.append(df_toronto1, ignore_index=True)

## Using Foursquare API to get venue information for neighborhoods

First, we define Foursquare credentials and version.

In [5]:
CLIENT_ID = '' # Foursquare ID, removed for sharing
CLIENT_SECRET = '' # your Foursquare Secret, removed for sharing
VERSION = '20180605' # Foursquare API version

And we define two functions to help extract information on venues.

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
# function that returns nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we run the above function on each neighborhood and create new dataframes called ny_venues and toronto_venues.

In [8]:
LIMIT = 50 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

venues = getNearbyVenues(names = df['Neighborhood'],
                                   latitudes = df['Latitude'],
                                   longitudes = df['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

Let's have a look at this new dataframe:

In [9]:
print(venues.shape)
venues.head()

(9602, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


## 3. Methodology and Analysis<a name="methodology"></a> &nbsp; <font size = 1>[return to top](#top)</font>

Having obtained the list of nearby venues for neighborhoods, we will use the **k-means clustering algorithm** to group them and identify similar neighborhoods in both cities. 

### Clustering neighborhoods

We start by preparing the dataframe.

In [10]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# change the name of the category named "Neighborhood"
onehot.rename(columns={'Neighborhood':'Neighborhood (venue category)'}, inplace=True)

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Ve

Now we group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [11]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Ve

Also, let's create a dataframe that displays the top 10 venues for each postal code.

In [12]:
# function that sorts the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Spa,Construction & Landscaping,Bus Station,Gas Station,Fried Chicken Joint,Breakfast Spot,Spanish Restaurant
1,Annadale,Diner,Sports Bar,Food,Park,Train Station,Bakery,Dance Studio,Restaurant,American Restaurant,Pizza Place
2,Arden Heights,Bus Stop,Pharmacy,Coffee Shop,Pizza Place,Farm,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
3,Arlington,Bus Stop,Intersection,Grocery Store,Deli / Bodega,American Restaurant,Coffee Shop,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Service
4,Arrochar,Deli / Bodega,Bagel Shop,Bus Stop,Italian Restaurant,Pizza Place,Food Truck,Supermarket,Middle Eastern Restaurant,Mediterranean Restaurant,Outdoors & Recreation


We are now ready to run k-means to cluster the neighborhoods. We don't want the number of clusters to be too small or large, so we will run the algorithm for values of k between 4 and 9 (not including 9), and evaluate the model accuracy using it's **silhouette score** to then choose the most adequate k.

In [14]:
grouped_clustering = grouped.drop('Neighborhood', 1)

# initialize maximum score at minimum possible value
max_score = -1

for kclusters in range(4,9):
    # runs k-means clustering
    kmeans_temp = KMeans(n_clusters = kclusters, random_state = 0).fit(grouped_clustering)
    # gets labels
    labels = kmeans_temp.labels_
    # calculates the silhouette score
    score = metrics.silhouette_score(grouped_clustering, labels, metric='euclidean')
    # if the score was better then previous scores, save it on max_score, and the value of k at best_k and the model at kmeans
    if max_score < score:
        max_score = score
        best_k = kclusters
        kmeans = kmeans_temp

print(f'The number of clusters is {best_k}.')

The number of clusters is 5.


And we create a new dataframe that includes the cluster label as well as the top 10 venues for each neighborhood.

In [15]:
# add clustering labels
neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = df

# merge grouped with df to add latitude/longitude for each neighborhood
merged = merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2.0,Food Truck,Pizza Place,Donut Shop,Sandwich Place,Pharmacy,Laundromat,Dessert Shop,Gas Station,Caribbean Restaurant,Ice Cream Shop
1,Bronx,Co-op City,40.874294,-73.829939,2.0,Baseball Field,Bus Station,Chinese Restaurant,Mattress Store,Discount Store,Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Park
2,Bronx,Eastchester,40.887556,-73.827806,2.0,Caribbean Restaurant,Bus Station,Metro Station,Deli / Bodega,Diner,Convenience Store,Juice Bar,Bowling Alley,Bus Stop,Fast Food Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,1.0,Plaza,River,Playground,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Bronx,Riverdale,40.890834,-73.912585,1.0,Park,Plaza,Bus Station,Bank,Gym,Locksmith,Home Service,Playground,Food Truck,Deli / Bodega


Finally, let's clean this dataframe we just obtained.

In [16]:
# drops rows which contain NaN
merged1 = merged.dropna().reset_index(drop = True)

# transforms 'Cluster Labels' column into int type
merged2 = merged1.astype({'Cluster Labels': int})

merged2.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Food Truck,Pizza Place,Donut Shop,Sandwich Place,Pharmacy,Laundromat,Dessert Shop,Gas Station,Caribbean Restaurant,Ice Cream Shop
1,Bronx,Co-op City,40.874294,-73.829939,2,Baseball Field,Bus Station,Chinese Restaurant,Mattress Store,Discount Store,Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Park
2,Bronx,Eastchester,40.887556,-73.827806,2,Caribbean Restaurant,Bus Station,Metro Station,Deli / Bodega,Diner,Convenience Store,Juice Bar,Bowling Alley,Bus Stop,Fast Food Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,1,Plaza,River,Playground,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Bronx,Riverdale,40.890834,-73.912585,1,Park,Plaza,Bus Station,Bank,Gym,Locksmith,Home Service,Playground,Food Truck,Deli / Bodega


Also, let's split this dataframe in two dataframes, one for each city.

In [17]:
# select rows which contain neighborhoods in Ney York
ny_merged = merged2.loc[merged2['Neighborhood'].isin(df_ny['Neighborhood'])]

ny_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Food Truck,Pizza Place,Donut Shop,Sandwich Place,Pharmacy,Laundromat,Dessert Shop,Gas Station,Caribbean Restaurant,Ice Cream Shop
1,Bronx,Co-op City,40.874294,-73.829939,2,Baseball Field,Bus Station,Chinese Restaurant,Mattress Store,Discount Store,Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Park
2,Bronx,Eastchester,40.887556,-73.827806,2,Caribbean Restaurant,Bus Station,Metro Station,Deli / Bodega,Diner,Convenience Store,Juice Bar,Bowling Alley,Bus Stop,Fast Food Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,1,Plaza,River,Playground,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Bronx,Riverdale,40.890834,-73.912585,1,Park,Plaza,Bus Station,Bank,Gym,Locksmith,Home Service,Playground,Food Truck,Deli / Bodega


In [18]:
# select rows which contain neighborhoods in Toronto
toronto_merged = merged2.loc[merged2['Neighborhood'].isin(df_toronto['PostalCode'])]

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
303,Scarborough,M1B,43.806686,-79.194353,2,Fast Food Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory
304,Scarborough,M1C,43.784535,-79.160497,1,Moving Target,Bar,Yoga Studio,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
305,Scarborough,M1E,43.763573,-79.188711,2,Breakfast Spot,Rental Car Location,Medical Center,Mexican Restaurant,Electronics Store,Pizza Place,Intersection,Food,Farm,English Restaurant
306,Scarborough,M1G,43.770992,-79.216917,1,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
307,Scarborough,M1H,43.773136,-79.239476,1,Bakery,Fried Chicken Joint,Hakka Restaurant,Bank,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Lounge,Falafel Restaurant,English Restaurant


## 4. Results and Discussion <a name="results"></a> &nbsp; <font size = 1>[return to top](#top)</font>

Now that we have clustered the neighborhoods, let's create maps to visualize the neighborhoods in their cities, labeled by their clusters.

First, let's set a color scheme for the clusters.

In [19]:
x = np.arange(best_k)
ys = [i + x + (i*x)**2 for i in range(best_k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

print('The hexadecimal codes of the colors in the scheme are: ', rainbow)

The hexadecimal codes of the colors in the scheme are:  ['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


Then create a map of New York.

In [20]:
# obtain coordinates for New York using geolocator
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'explorer')
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude

# create map
map_clusters_ny = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_ny)
       
map_clusters_ny

And now the map for Toronto.

In [21]:
# obtain coordinates for Toronto using geolocator
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = 'explorer')
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude

# create map
map_clusters_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_toronto)
       
map_clusters_toronto

Finally, let's look which are top 5 most common venues for each cluster.

In [22]:
# insert cluster labels on grouped dataframe
grouped.insert(0, 'Cluster Labels', kmeans.labels_)

# group by clusters
clusters_grouped = grouped.groupby(['Cluster Labels']).sum()

# number of top categories we will consider
num_top_cat = 5

for i in range(best_k):
    # use return_most_common_venues function to get most common venues
    top_cat = return_most_common_venues(clusters_grouped.iloc[i], num_top_cat)
    # print most common venues on this cluster
    print(f'The most common venue categories in cluster {i} are: ', end = '')
    for category in top_cat:
        if top_cat[num_top_cat - 1] == category:
            print(category, end = '.')
        else:
            print(category, end = ', ')
    print()

The most common venue categories in cluster 0 are: Bus Stop, Pizza Place, Park, Bagel Shop, Coffee Shop.
The most common venue categories in cluster 1 are: Coffee Shop, Café, Italian Restaurant, Bar, Park.
The most common venue categories in cluster 2 are: Pizza Place, Deli / Bodega, Pharmacy, Bank, Donut Shop.
The most common venue categories in cluster 3 are: Deli / Bodega, Beach, Pier, Beach Bar, Athletics & Sports.
The most common venue categories in cluster 4 are: Park, Playground, Fast Food Restaurant, Pizza Place, River.


## 5. Discussion <a name="discussion"></a> &nbsp; <font size = 1>[return to top](#top)</font>

The data provided by the maps is easily understood, so recommendations based on which regions of both cities are most similar to one another are straightforwardly provided by them. So now let's examine possible limitations to our analysis.

Firstly, there are various features other than venues' categories that could be used to group the neighborhoods, although they might be harder to find extensive and quantitative data for.

However, let's see possible limitations to the data we used here.

Let's start by checking the number of venues returned by neighborhood.

In [23]:
# add a column containing 1 for each venue
venues['Venue count'] = 1

# use the column we created to group by neighborhood and obtain the number of venues returned per neighborhood
venues_count = venues[['Neighborhood', 'Venue count']].groupby(by = 'Neighborhood').sum()

venues_count.head()

,Venue count
Neighborhood,
Allerton,27
Annadale,10
Arden Heights,4
Arlington,6
Arrochar,20


As you can see, there seems to be an imbalance in the number of venues returned by neighborhood. To better visualize this, let's see the average number of venues returned for neighborhoods by borough.

In [24]:
# selects only boroughs and neighborhoods, sorts by neighborhood then reset index
boroughs = df[['Borough', 'Neighborhood']].sort_values(by = 'Neighborhood').reset_index(drop = True)

boroughs.head()

,Borough,Neighborhood
0,Bronx,Allerton
1,Staten Island,Annadale
2,Staten Island,Arden Heights
3,Staten Island,Arlington
4,Staten Island,Arrochar


In [25]:
# join the two previously created dataframes
venues_borough = boroughs.join(venues_count, on='Neighborhood')

venues_borough.head()

,Borough,Neighborhood,Venue count
0,Bronx,Allerton,27.0
1,Staten Island,Annadale,10.0
2,Staten Island,Arden Heights,4.0
3,Staten Island,Arlington,6.0
4,Staten Island,Arrochar,20.0


In [26]:
# check the average number of venues returned for neighborhoods by borough
venues_borough.groupby(by = ['Borough']).mean()

,Venue count
Borough,
Bronx,21.692308
Brooklyn,30.542857
Central Toronto,12.888889
Downtown Toronto,42.277778
East Toronto,23.800000
East York,15.200000
Etobicoke,6.636364
Manhattan,48.550000
Mississauga,11.000000


On the other hand, that suggests that having our analysis in Toronto be based on postal codes instead of neighborhoods may have helped reduce this imbalance.

Despite the aforementioned limitations on data, altogether we believe our analysis can still provide helpful insights to those considering to move between both cities.

## 6. Conclusion <a name="conclusion"></a> &nbsp; <font size = 1>[return to top](#top)</font>

The purpose of this project was to cluster neighborhoods in New York and Toronto into groups with similar characteristics. To do so, we used Foursquare API to get information on venues and then used the k-means clustering algorithm to group neighborhoods according to venue category, choosing the most adequate value of k in a certain range. In the end, we visualized the distribution of neighborhoods, labeled by their clusters, in a map of each city.

Although there are several factors to consider when moving between New York and Toronto, the analysis we conducted here can aid that decision by showing which areas of both cities are most similar to one another.